In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import gspread 
import requests
import dash
from dash import dcc, html, dash_table

pd.set_option('display.expand_frame_repr', False)  # Prevent line wrapping

In [19]:
gc = gspread.service_account(filename="service_account.json")
contestbeta = gc.open("2024 Playoffs - Wild Card (Responses)")
pickinput = contestbeta.worksheet("Form Responses 1")
picksraw = pd.DataFrame(pickinput.get_all_records())

In [6]:
# Rename columns, drop email and notes
picks = picksraw.set_axis(
    ['timestamp', 'email', 'name', 'afc1', 'afc1con', 'afc2', 'afc2con', 'afc3', 'afc3con',
     'nfc1', 'nfc1con', 'nfc2', 'nfc2con', 'nfc3', 'nfc3con', 'notes'],
    axis=1
).drop(columns=['email', 'notes'])

# Convert timestamp to datetime for sorting
picks['timestamp'] = pd.to_datetime(picks['timestamp'])

# Define the shared deadlines (first kickoff for each day)
shared_deadlines = {
    'saturday': pd.Timestamp('2025-01-11 13:30:00'),  # First Saturday game
    'sunday': pd.Timestamp('2025-01-12 10:00:00')     # First Sunday game (applies to Monday too)
}

# Map each game to the correct deadline
game_deadlines = {
    'afc1': shared_deadlines['saturday'],
    'afc2': shared_deadlines['saturday'],
    'afc3': shared_deadlines['sunday'],
    'nfc1': shared_deadlines['sunday'],
    'nfc2': shared_deadlines['sunday'],
    'nfc3': shared_deadlines['sunday']  # Monday game but same as Sunday deadline
}

# Pivot to long format, preserving multiple picks
picks_long = pd.melt(picks, 
                     id_vars=['timestamp', 'name'], 
                     value_vars=['afc1', 'afc2', 'afc3', 'nfc1', 'nfc2', 'nfc3'],
                     var_name='game', 
                     value_name='pick')

conf_long = pd.melt(picks,
                    id_vars=['timestamp', 'name'], 
                    value_vars=['afc1con', 'afc2con', 'afc3con', 'nfc1con', 'nfc2con', 'nfc3con'],
                    var_name='game', 
                    value_name='confidence')

# Clean confidence column names to match picks
conf_long['game'] = conf_long['game'].str.replace('con', '')

# Merge picks with confidence
df_merged = pd.merge(picks_long, conf_long, on=['timestamp', 'name', 'game'])

# Assign deadline based on the game
df_merged['deadline'] = df_merged['game'].map(game_deadlines)

# Flag late picks
df_merged['late'] = df_merged['timestamp'] > df_merged['deadline']

# Sort 
df_merged.sort_values(by=['game', 'name', 'timestamp'], inplace=True)

print(df_merged.head(15))


             timestamp          name  game    pick confidence            deadline   late
4  2024-01-12 21:24:40        Adam W  afc1  Browns         12 2025-01-11 13:30:00  False
16 2024-01-13 10:48:24        Alan B  afc1  Browns         10 2025-01-11 13:30:00  False
32 2024-01-14 12:51:49        Alan B  afc1                    2025-01-11 13:30:00  False
21 2024-01-13 13:11:19        Austin  afc1  Browns          3 2025-01-11 13:30:00  False
33 2024-01-14 13:01:46        Austin  afc1                    2025-01-11 13:30:00  False
8  2024-01-13 07:57:15    BiderErnst  afc1  Texans          4 2025-01-11 13:30:00  False
27 2024-01-14 09:02:05    BiderErnst  afc1                    2025-01-11 13:30:00  False
41 2024-01-14 14:09:39    BiderErnst  afc1                    2025-01-11 13:30:00  False
23 2024-01-13 13:34:48          Blue  afc1  Browns          2 2025-01-11 13:30:00  False
36 2024-01-14 13:29:42          Blue  afc1                    2025-01-11 13:30:00  False
5  2024-01-12 21:32:1

In [7]:
# Step 1: Filter to only picks made before the deadline
valid_picks = df_merged[df_merged['timestamp'] <= df_merged['deadline']]

# Step 2: Drop rows with blank picks (ensuring they are removed before selecting latest)
valid_picks = valid_picks[valid_picks['pick'] != '']

# Step 3: Keep the latest pick (by timestamp) for each player and game
valid_picks = valid_picks.sort_values(by=['timestamp']).groupby(['name', 'game']).tail(1)

# Step 4: Remove rows where confidence is NaN and ensure confidence values are integers
valid_picks = valid_picks.dropna(subset=['confidence'])
valid_picks = valid_picks[valid_picks['confidence'].apply(lambda x: isinstance(x, int))]

# Step 5: Define the game order
game_order = {'afc1': 1, 'afc2': 2, 'afc3': 3, 'nfc1': 4, 'nfc2': 5, 'nfc3': 6}

# Step 6: Add game order and initialize dupval
valid_picks['game_order'] = valid_picks['game'].map(game_order)
valid_picks['dupval'] = None  # Initialize with None, not np.nan

# Step 7: Sort by player and game order
valid_picks = valid_picks.sort_values(by=['name', 'game_order'])

# Step 8: Zero out duplicates and track them (Direct DataFrame Iteration)
used_conf = {}

for idx, row in valid_picks.iterrows():
    player = row['name']
    conf = row['confidence']

    if player not in used_conf:
        used_conf[player] = set()

    if conf in used_conf[player]:
        # Zero out the confidence and store the duplicated value in dupval
        valid_picks.at[idx, 'confidence'] = 0
        valid_picks.at[idx, 'dupval'] = conf
    else:
        used_conf[player].add(conf)

# Step 9: Drop the temporary game_order column
valid_picks = valid_picks.drop(columns=['game_order'])

# Step 10: Display results for Ryan B
print(valid_picks.loc[valid_picks.name == 'Ryan B'])


              timestamp    name  game        pick confidence            deadline   late dupval
6   2024-01-13 06:24:08  Ryan B  afc1      Browns          8 2025-01-11 13:30:00  False   None
50  2024-01-13 06:24:08  Ryan B  afc2    Dolphins          9 2025-01-11 13:30:00  False   None
94  2024-01-13 06:24:08  Ryan B  afc3       Bills          6 2025-01-12 10:00:00  False   None
138 2024-01-13 06:24:08  Ryan B  nfc1     Packers         10 2025-01-12 10:00:00  False   None
182 2024-01-13 06:24:08  Ryan B  nfc2       Lions         11 2025-01-12 10:00:00  False   None
226 2024-01-13 06:24:08  Ryan B  nfc3  Buccaneers          0 2025-01-12 10:00:00  False      6


In [8]:
# CONFIDENCE TRACKING

# Step 1: Prepare a list of original confidence values by replacing zeros with dupval
valid_picks['adjusted_conf'] = valid_picks.apply(
    lambda row: row['confidence'] if row['confidence'] != 0 else row['dupval'], axis=1
)

# Step 2: Group by player and collect sorted adjusted confidence values into a list
player_confidence = (
    valid_picks.groupby('name')['adjusted_conf']
    .apply(lambda x: sorted(list(x)))
    .reset_index(name='entered_conf')
)

# Step 3: Create effective_conf by zeroing out duplicates (keeping first occurrence)
def zero_duplicates(conf_list):
    seen = set()
    effective = []
    
    for conf in conf_list:
        if conf in seen:
            effective.append(0)  # Zero out duplicates
        else:
            effective.append(conf)
            seen.add(conf)
    
    return effective

player_confidence['effective_conf'] = player_confidence['entered_conf'].apply(zero_duplicates)

# Step 4: Calculate remaining confidence values

def calculate_remaining_conf(effective_conf):
    all_values = set(range(1, 14))  # Full set of confidence values (1 to 13)
    remaining = sorted(all_values - set(effective_conf))

    for conf in effective_conf:
        if conf == 0 and remaining:
            remaining.pop(0)  # Burn the lowest remaining value for each zero
    
    return remaining

player_confidence['remaining_conf'] = player_confidence['effective_conf'].apply(calculate_remaining_conf)

TOTAL_GAMES=6
misspicks = valid_picks.groupby('name')['game'].count().reset_index(name='picks_made')
misspicks['missed_picks'] = TOTAL_GAMES - misspicks['picks_made']
player_confidence = player_confidence.merge(misspicks[['name', 'missed_picks']], on='name', how='left')

# Step 6: Adjust remaining confidence values based on missed picks

def adjust_remaining_for_missed_picks(row):
    remaining = row['remaining_conf'][:]
    missed = row['missed_picks']
    
    for _ in range(missed):
        if remaining:
            median_idx = (len(remaining) - 1) // 2
            remaining.pop(median_idx)  # Remove the median value for each missed pick

    return remaining

player_confidence['remaining_conf'] = player_confidence.apply(adjust_remaining_for_missed_picks, axis=1)

print(player_confidence)

            name            entered_conf          effective_conf             remaining_conf  missed_picks
0         Adam W    [3, 4, 6, 9, 11, 12]    [3, 4, 6, 9, 11, 12]    [1, 2, 5, 7, 8, 10, 13]             0
1         Alan B    [3, 4, 6, 8, 10, 11]    [3, 4, 6, 8, 10, 11]    [1, 2, 5, 7, 9, 12, 13]             0
2         Austin     [2, 3, 5, 7, 9, 10]     [2, 3, 5, 7, 9, 10]   [1, 4, 6, 8, 11, 12, 13]             0
3     BiderErnst    [2, 3, 4, 4, 11, 13]    [2, 3, 4, 0, 11, 13]    [5, 6, 7, 8, 9, 10, 12]             0
4           Blue      [2, 4, 5, 6, 7, 8]      [2, 4, 5, 6, 7, 8]  [1, 3, 9, 10, 11, 12, 13]             0
5        Brian R      [3, 4, 5, 6, 8, 9]      [3, 4, 5, 6, 8, 9]  [1, 2, 7, 10, 11, 12, 13]             0
6         Clouse     [1, 3, 4, 9, 9, 13]     [1, 3, 4, 9, 0, 13]   [5, 6, 7, 8, 10, 11, 12]             0
7   Giusto/Simon      [1, 3, 4, 5, 8, 9]      [1, 3, 4, 5, 8, 9]  [2, 6, 7, 10, 11, 12, 13]             0
8        Goldich     [1, 3, 5, 7, 9, 11]     [

In [9]:
# IMPORT GAME RESULTS
playoffs_wc = gc.open("2024 Playoffs - Wild Card (Responses)")
wc_lines_scores = playoffs_wc.worksheet("lines_scores")
wc_results = pd.DataFrame(wc_lines_scores.get_all_records())
wc_results

,game,away,home,homeline,awaypts,homepts,winner_ATS,complete
0,afc1,Browns,Texans,-2.5,14,45,Texans,1
1,afc2,Dolphins,Chiefs,4.5,7,26,Chiefs,1
2,afc3,Steelers,Bills,9.5,17,31,Bills,1
3,nfc1,Packers,Cowboys,7.0,48,32,Packers,1
4,nfc2,Rams,Lions,3.0,23,24,Lions,1
5,nfc3,Eagles,Buccaneers,-3.0,9,32,Buccaneers,1


In [10]:
# SCORE PICKS

# Step 2: Merge results with player picks (ensure not to remerge if already done)
if 'winner_ATS' not in valid_picks.columns:
    valid_picks = valid_picks.merge(wc_results[['game', 'winner_ATS', 'complete']], on='game', how='left')

# Step 3: Score picks - Compare player picks to the winner
valid_picks['correct'] = (valid_picks['pick'] == valid_picks['winner_ATS']).astype(int)

# Step 4: Calculate points won (without function)
valid_picks['points_won'] = valid_picks['confidence'] * (
    (valid_picks['winner_ATS'] == 'Push') * 0.5 +  # Half points for push
    (valid_picks['correct'])                       # Full points for correct picks
)

# Step 5: Optional - Filter only completed games
valid_picks = valid_picks[valid_picks['complete'] == 1]

valid_picks


,timestamp,name,game,pick,confidence,deadline,late,dupval,adjusted_conf,winner_ATS,complete,correct,points_won
0,2024-01-12 21:24:40,Adam W,afc1,Browns,12,2025-01-11 13:30:00,False,None,12,Texans,1,0,0.0
1,2024-01-12 21:24:40,Adam W,afc2,Chiefs,4,2025-01-11 13:30:00,False,None,4,Chiefs,1,1,4.0
2,2024-01-12 21:24:40,Adam W,afc3,Bills,9,2025-01-12 10:00:00,False,None,9,Bills,1,1,9.0
3,2024-01-12 21:24:40,Adam W,nfc1,Cowboys,11,2025-01-12 10:00:00,False,None,11,Packers,1,0,0.0
4,2024-01-12 21:24:40,Adam W,nfc2,Rams,3,2025-01-12 10:00:00,False,None,3,Lions,1,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
132,2024-01-13 12:52:16,Woody,afc2,Chiefs,13,2025-01-11 13:30:00,False,None,13,Chiefs,1,1,13.0
133,2024-01-14 13:13:24,Woody,afc3,Steelers,9,2025-01-12 10:00:00,False,None,9,Bills,1,0,0.0
134,2024-01-14 13:13:24,Woody,nfc1,Packers,5,2025-01-12 10:00:00,False,None,5,Packers,1,1,5.0
135,2024-01-14 13:13:24,Woody,nfc2,Lions,4,2025-01-12 10:00:00,False,None,4,Lions,1,1,4.0


In [11]:
# PREPARE DATA FOR DASH DISPLAY

# 1. Game Info Table (from wc_results)
game_info = wc_results[['game', 'away', 'home', 'homeline', 'awaypts', 'homepts', 'winner_ATS']]

# 2. Summary Table of Player Scores
player_scores = valid_picks.groupby('name')['points_won'].sum().reset_index(name='total_points')

# Merge remaining confidence values from player_confidence DataFrame
player_scores = player_scores.merge(
    player_confidence[['name', 'remaining_conf']], 
    on='name', 
    how='left'
)

# Convert list to string for display
player_scores['remaining_conf'] = player_scores['remaining_conf'].apply(lambda x: ', '.join(map(str, x)))
# 3. Full Picks Results Table
picks_results = valid_picks[['name', 'game', 'pick', 'confidence', 'points_won']]

# Initialize Dash App
app = dash.Dash(__name__)

# DASH LAYOUT WITH ALL TABLES
app.layout = html.Div([
    html.H1("NFL Playoff Contest Results"),

    html.H2("Game Results"),
    dash_table.DataTable(
        data=game_info.to_dict('records'),
        columns=[{"name": i, "id": i} for i in game_info.columns],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'center', 'padding': '10px'},
        style_header={'backgroundColor': 'lightblue', 'fontWeight': 'bold'}
    ),

    html.H2("Player Score Summary"),
    dash_table.DataTable(
        data=player_scores.to_dict('records'),
        columns=[{"name": i, "id": i} for i in player_scores.columns],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'center', 'padding': '10px'},
        style_header={'backgroundColor': 'lightblue', 'fontWeight': 'bold'}
    ),

    html.H2("Player Picks and Points"),
    dash_table.DataTable(
        data=picks_results.to_dict('records'),
        columns=[{"name": i, "id": i} for i in picks_results.columns],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'center', 'padding': '10px'},
        style_header={'backgroundColor': 'lightblue', 'fontWeight': 'bold'}
    )
])


In [12]:
if __name__ == '__main__':
    app.run_server(debug=True)